In [ ]:
!pip install datasets
!pip install transformers
!pip install torch

In [2]:
from datasets import load_dataset,DatasetDict,Dataset
from transformers import BertTokenizer,BertForSequenceClassification
from torch.utils.data import DataLoader
from transformers import Trainer,TrainingArguments,DataCollatorWithPadding
import torch
import logging

logging.disable(logging.WARNING)

In [5]:
df = load_dataset('csv',data_files='spam.csv',encoding='latin-1')

In [6]:
df = df.remove_columns(['Unnamed: 2','Unnamed: 3','Unnamed: 4'])

In [7]:
df=df.rename_columns({'v1':'label','v2':'text'})
df = df.class_encode_column('label')

Casting to class labels:   0%|          | 0/5572 [00:00<?, ? examples/s]

In [8]:
shuffeled = df.shuffle(seed=42)

In [9]:
train_test_split= shuffeled['train'].train_test_split(test_size=0.2)

In [10]:
shuffeled_train= train_test_split['train']
shuffeled_test = train_test_split['test']

In [11]:
to_tokenize = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [12]:
def tokenize(content):
  return to_tokenize(content['text'],padding='max_length',truncation=True)

In [13]:
shuffeled_train = shuffeled_train.map(tokenize,batched=True,batch_size=400,num_proc=4)
shuffeled_test = shuffeled_test.map(tokenize,batched=True,batch_size=400,num_proc=4)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/4457 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1115 [00:00<?, ? examples/s]

In [14]:
shuffeled_train_loader = DataLoader(shuffeled_train,batch_size=8,shuffle=True)
shuffeled_test_loader = DataLoader(shuffeled_test,batch_size=8)

In [21]:
my_model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
training_arguments= TrainingArguments(
        output_dir='./eva_results',
        num_train_epochs=3,
        per_device_train_batch_size=2,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_dir='./train_logs',
        logging_steps=10,
        dataloader_num_workers=1,
        gradient_accumulation_steps=2
)

In [22]:
model_trainer= Trainer(
    model=my_model,
    args=training_arguments,
    train_dataset=shuffeled_train,
    eval_dataset=shuffeled_test,
    tokenizer=to_tokenize,
    data_collator=DataCollatorWithPadding(tokenizer=to_tokenize)
)

In [24]:
model_trainer.train()

Epoch,Training Loss,Validation Loss
0,0.001200,0.054890
2,0.000400,0.045169


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


TrainOutput(global_step=3342, training_loss=0.08107021214936641, metrics={'train_runtime': 1498.8086, 'train_samples_per_second': 8.921, 'train_steps_per_second': 2.23, 'total_flos': 3516742365941760.0, 'train_loss': 0.08107021214936641, 'epoch': 2.9986541049798117})

In [25]:
evaluation_results=model_trainer.evaluate()
print(evaluation_results)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


{'eval_loss': 0.045168764889240265, 'eval_runtime': 30.7594, 'eval_samples_per_second': 36.249, 'eval_steps_per_second': 4.551, 'epoch': 2.9986541049798117}


In [26]:
def predict(text):
    inputs = to_tokenize(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(my_model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = my_model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
    return "AI-generated" if predictions.item() == 1 else "Human-written"

example_text ="Python is a versatile programming language known for its simplicity and readability. It's widely used in web development, data analysis, artificial intelligence, and more. Its extensive libraries and supportive community make it an excellent choice for both beginners and experienced developers."
print(predict(example_text))

AI-generated
